Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
import numpy as np
from copy import deepcopy
from icecream import ic

In [111]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [114]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) ## valori degli oggetti
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) ## pesi degli oggetti (n.b ogni peso ha più dimensioni e i constraint devono essere rispettati in ogni dimensione)
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) ## tante quante le dimensioni 

In [115]:
CONSTRAINTS
# ogni zaino è una riga e ogni colonna è il constraint sulla dimensione

array([[235, 442],
       [184, 200]], dtype=int32)

In [5]:
def flip_item(sol):
    new_sol = np.copy(sol)
    k = np.random.randint(NUM_KNAPSACKS)
    i = np.random.randint(NUM_ITEMS)
    new_sol[k, i] = not new_sol[k, i]
    return new_sol

def move_item(sol):
    new_sol = np.copy(sol)
    i = np.random.randint(NUM_ITEMS)
    from_k = np.argmax(sol[:, i]) if np.any(sol[:, i]) else None
    to_k = np.random.randint(NUM_KNAPSACKS)

    if from_k is not None:
        new_sol[from_k, i] = False
    new_sol[to_k, i] = True
    return new_sol

def swap_items(sol):
    new_sol = np.copy(sol)
    k1, k2 = np.random.choice(NUM_KNAPSACKS, 2, replace=False)
    i1, i2 = np.random.randint(NUM_ITEMS, size=2)
    new_sol[k1, i1], new_sol[k2, i2] = new_sol[k2, i2], new_sol[k1, i1]
    return new_sol


In [6]:
def tweak(solution):
    new_solution = np.copy(solution)

    # scegli casualmente un'operazione di modifica
    move_type = np.random.choice(["flip", "move", "swap"], p=[0.5, 0.3, 0.2])
    
    if move_type == "flip":
        new_solution = flip_item(new_solution)
    elif move_type == "move":
        new_solution = move_item(new_solution)
    elif move_type == "swap":
        new_solution = swap_items(new_solution)
    
    return new_solution


In [3]:
def enforce_unique_assignment(sol):
    # se un item è in più knapsack, lascialo in uno solo
    for i in range(NUM_ITEMS):
        assigned = np.where(sol[:, i])[0]
        if len(assigned) > 1:
            keep = np.random.choice(assigned)
            sol[:, i] = False
            sol[keep, i] = True
    return sol

In [92]:
# fitness function: higher is better
def fitness_cost(solution: list[set] ) -> (float, float):
    total_value = 0
    penalty = 0.0

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]

        total_value += knapsack_value

        # penalità proporzionale all'eccesso
        overweight = knapsack_weight - CONSTRAINTS[b]
        penalty += np.sum(np.maximum(0, overweight))

    
    # il parametro di bilanciamento dinamico lo aumento, inizialmente è più piccolo per permettere 
    # l'esplorazione poi lo aumento per penalizzare/escludere soluuzioni non valide
    alpha =  10
    # fitness = valore totale - penalità pesata
    fitness = total_value - alpha * penalty

    return (fitness, penalty)


In [116]:

current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True
alpha_start = 1
alpha_end = 50
MAX_STEPS = 500

current_fitness, current_penality = fitness_cost(current_solution)




for step in range(MAX_STEPS):

   # ic(current_solution, current_cost)
    new_solution= tweak(current_solution)
    new_solution= enforce_unique_assignment(new_solution)
    new_fitness, new_penality = fitness_cost(new_solution)
    if new_fitness> current_fitness: #cost in realtà è una fitness function
        current_solution= new_solution
        current_fitness= new_fitness
        current_penality= new_penality

current_solution
current_fitness
        

np.float64(345.0)

In [94]:
solution= current_solution
solution

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, Fals

In [121]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.False_

In [96]:
# Check if the solution is valid
#all_knapsacks = np.any(solution, axis=0)
#np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

valid_per_bag = []
for b in range(NUM_KNAPSACKS):
    # seleziona gli item presenti nello zaino b
    items_in_b = np.where(solution[b])[0]            # indici degli item presenti
    total_weight = WEIGHTS[items_in_b].sum(axis=0)   # somma per dimensione
    valid_per_bag.append(np.all(total_weight <= CONSTRAINTS[b]))

overall_valid = all(valid_per_bag)
overall_valid

False

## TEST PROBLEMS

In [43]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[168, 185],
       [196, 553],
       [ 38, 428]], dtype=int32)

In [15]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[2627, 7071, 7260, 1477, 1437, 1552, 1557, 5519, 4268, 6845],
       [2283, 2697, 6398, 6820,  810, 8570, 7727, 5063, 3517, 5747],
       [9130,  568, 9155, 3689,  631, 1136,  663, 6094, 8960, 9278],
       [2541, 2772, 9810, 2564, 8566, 2830, 4443, 5588, 1213, 8433],
       [7828, 1639, 6619, 8434, 4514, 4476, 4785, 1655, 5846, 8557],
       [3561, 8030, 5553, 7149,  844, 2556, 5460, 6295, 9945, 8043],
       [9545,  413, 1059, 8712, 9278, 4840, 2541, 6571, 9832, 6896],
       [3717, 6932,   59, 2288, 1264, 7220, 2869, 7139, 4680, 4419],
       [9247, 3582, 1999, 4638, 4140, 2685, 3392, 9691, 9536,  831],
       [2843, 9815, 3902, 3731, 7797, 8896, 9080, 8919,  337, 7041]],
      dtype=int32)

In [32]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS


array([[30952, 61599, 63873, ..., 40668, 56940, 36914],
       [90162, 42279, 90348, ..., 19966, 80524, 20075],
       [89828, 67367, 24693, ..., 18613, 17203, 71643],
       ...,
       [48577, 29359, 83956, ..., 52110, 18853, 52126],
       [25488, 46553, 88217, ..., 43390, 39936, 62540],
       [17638, 31657, 87646, ..., 27780, 86129, 53404]],
      shape=(100, 100), dtype=int32)

OLD SOLUTION 

In [ ]:
#### OLD SOLUTION# --- IGNORE ---

def tweak_non_valida(knapsacs:  list[set] ) -> list[set]:
    ## conviene togliere un oggetto da uno zaino che è in sovrappeso
    new_bags = deepcopy(knapsacs)
    # trova zaini in sovrappeso
    weights_per_bag = new_bags.astype(int).dot(WEIGHTS)
    overweight_mask = weights_per_bag > CONSTRAINTS
    overweight_bags = np.where(np.any(overweight_mask, axis=1))[0]

    for b in overweight_bags:
        ## IDEA: al posto di togliere uno zaino casuale possiamo togliere quello con la componente più alta che sfora
        ## excess = np.maximum(0, weights_per_bag[b] - CONSTRAINTS[b])
        items_in_bag = np.where(new_bags[b])[0]
        
        if len(items_in_bag) > 0:
            item_to_remove = np.random.choice(items_in_bag)
            new_bags[b][item_to_remove] = False
            
    return new_bags

def tweak_valida(knapsack: list[set]) -> list[set]:
    ## conviene aggiungere un oggetto a uno zaino che non è in sovrappeso
    new_bags = deepcopy(knapsack)
    weights_per_bag = new_bags.astype(int).dot(WEIGHTS)

    # Calcola spazio residuo per ogni zaino valido
    remaining_capacity = np.maximum(0, CONSTRAINTS - weights_per_bag)
    # Scegli quello con margine totale più alto
   # remaining_capacity: (NUM_KNAPSACKS, NUM_DIMENSIONS)
    b = np.argmax(np.sum(remaining_capacity, axis=1))


    # Margine residuo
    current_weight = weights_per_bag[b]
    margin = CONSTRAINTS[b] - current_weight

    # Oggetti già presenti e mancanti
    items_in_bag = np.where(new_bags[b])[0]
    items_not_in_bag = np.where(~new_bags[b])[0]

     # Trova oggetti che entrano nei vincoli
    feasible_items = [i for i in items_not_in_bag if np.all(WEIGHTS[i] <= margin)]
    if len(feasible_items) > 0:
        item_to_add = np.random.choice(feasible_items)
        new_bags[b][item_to_add] = True
        return new_bags
    else:
        ## se nessun oggetto entra provo a scambiare
        i_remove = np.random.choice(items_in_bag)
        i_add = np.random.choice(items_not_in_bag)
        # new_weight = current_weight - WEIGHTS[i_remove] + WEIGHTS[i_add]
        new_bags[b][i_remove] = False
        new_bags[b][i_add] = True
    return new_bags

#OLD SOLUTION
## il costo di una soluzione è dato dalla somma dei valori degli oggetti meno la somma dei sovrappesi
# se la soluzione è valida il costo è negativo
def cost(solution: list[set]) -> (float, float, bool):

    weights_per_bag = solution.astype(int).dot(WEIGHTS)  
    total_overweight = weights_per_bag - CONSTRAINTS
    total_value = np.sum(solution.astype(int).dot(VALUES))
    # validità: nessuna componente > 0
    valid = not np.any(total_overweight > 0)

    # penalità: somma delle eccedenze
    penalty = np.sum(np.maximum(0, total_overweight))
    # fitness: valore - penalità pesata
    alpha = 10
    fitness = total_value - alpha * penalty

    return fitness, penalty, valid



current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True

(current_fitness, current_cost, current_valid) = cost(current_solution)


MAX_STEPS = 1000


for steps in range(MAX_STEPS):
    # left_items= np.where(~np.any(solution, axis=0))[0]  # item non ancora usati
    # if len(left_items==0):
    #     break
    ic(steps, current_cost, current_fitness, current_valid)
    if current_valid:
        ## soluzione valida
        
        new_solution = tweak_valida(current_solution)
        new_solution = enforce_unique_assignment(new_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)
        if new_valid:
            if new_fitness>= current_fitness: 
                ## la nuova soluzione è valida e il costo si avvicina a zero sempre di più (costo ideale)
                current_cost= new_cost
                current_fitness= new_fitness
                current_solution = new_solution
                current_valid= new_valid
                continue

    else: 
        ## soluzione non valida
        new_solution = tweak_non_valida(current_solution)
        new_solution = enforce_unique_assignment(new_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)

        if not new_valid: 
            
        ## la nuova soluzione non è valida anche essa 
            if new_cost < current_cost or (new_cost== current_cost and new_fitness>=current_fitness): 
                # la nuova soluzione si avvicina a essere valida 
                current_cost = new_cost
                current_solution = new_solution
                current_fitness= new_fitness
                current_valid= new_valid
                continue
            else:
                continue
        else:
            ## la nuova soluzione è valida 
            current_cost = new_cost
            current_solution = new_solution
            current_fitness= new_fitness
            current_valid= new_valid
            continue

    

print(current_cost, current_fitness, current_valid)



ic| steps: 0
    current_cost: np.int64(339)
    current_fitness: np.int64(-2979)
    current_valid: False
ic| steps: 1
    current_cost: np.int64(247)
    current_fitness: np.int64(-2158)
    current_valid: False
ic| steps: 2
    current_cost: np.int64(116)
    current_fitness: np.int64(-965)
    current_valid: False
ic| steps: 3
    current_cost: np.int64(54)
    current_fitness: np.int64(-403)
    current_valid: False
ic| steps: 4
    current_cost: np.int64(0)
    current_fitness: np.int64(109)
    current_valid: True


IndexError: boolean index did not match indexed array along axis 1; size of axis is 2 but size of corresponding boolean axis is 20

In [45]:
solution=current_solution
solution

array([[False,  True, False,  True, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
         True, False],
       [False, False,  True, False, False,  True, False, False, False,
        False, False,  True, False,  True, False,  True, False, False,
        False,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False,  True,
        False, False]])

In [46]:
# Check if the solution is valid
valid_per_bag = []
for b in range(NUM_KNAPSACKS):
    # seleziona gli item presenti nello zaino b
    items_in_b = np.where(solution[b])[0]            # indici degli item presenti
    total_weight = WEIGHTS[items_in_b].sum(axis=0)   # somma per dimensione
    valid_per_bag.append(np.all(total_weight <= CONSTRAINTS[b]))

overall_valid = all(valid_per_bag)
overall_valid

True

In [47]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.True_

In [48]:
def solution_evaluation(solution):
    total_value = 0
    percentage_weight_over_constraint = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]
        total_value += knapsack_value
        percentage_weight_over_constraint[b] = knapsack_weight*100/CONSTRAINTS[b]


    return percentage_weight_over_constraint, total_value

solution_evaluation(solution)

(array([[89.28571429, 81.08108108],
        [97.44897959, 75.22603978],
        [92.10526316, 28.03738318]]),
 np.int32(499))